# Mapping Waste Assets at NYCHA Developments using QGIS Python Scripting Capabilities

In the summer and fall of 2020, NYCHA's
									Strategic Planning Department worked to
									created Individualized Waste Management
									Action Plans (IWMAPs) for each of the
									Authority's 130+ "consolidations" (that is,
									a development or group of developments
									overseen by a single property management
									office). I was charged with several tasks,
									including, creating a map of waste assets
									at each consolidation for inclusion in the
									Action Plans. 

This document describes the process of
									creating those maps. Other work on the
									IWMAPs is presented on separate pages, or
									is in preparation.

### Tools and Methods

Due to restrictions on technology access
							associated with remote work (this was, after all,
							summer/fall 2020), all maps included in the reports
							were created using the open-source software QGIS.
							Fortunately, this was at the time (and remains) my
							preferred GIS software platform for its
							extensibility and versatility.

This project made particularly heavy use of
							QGIS' Python scripting capabilities. In fact,
							aside from basic layers-management tasks, all
							relevant work was completed using Python
							scripting.

The script used is displayed below.

In [ ]:
from qgis.core import *
from qgis.PyQt.QtGui import *
from qgis.PyQt.QtCore import *
from qgis.gui import *
import qgis.utils
import os
from os import path
import os.path
import sys

DEVS_LAYER_ID = 'NYCHA_Developments_b67e848b_1fce_439f_a808_806bac98ee07'
BUILDINGS_LAYER_ID = 'NYCHA_Buildings_cc15e63a_3f81_480c_b206_6b95bc37b83f'
ASSETS_LAYER_ID = 'Waste_Assets_1_5e60bec7_5309_46dd_8721_d94575f3629c'
BASEMAP_ID = 'Skobbler_3fb61fc5_38bc_464d_90b4_e4ad3b579c82'
RECYCLING_BINS_LAYER_ID = 'OUTPUT_65d16aa4_b789_491c_a639_9c73c3f3dd87'
ENTRANCES_LAYER_ID = 'Building_Entrances_27a1364c_03ab_47ac_9e78_11799a19994a'

processing.run("native:createspatialindex", {'INPUT':'/Users/kyleslugg/Documents/GIS/NYCHA/Waste_Assets/Waste_Assets_with_CID.shp|layername=Waste_Assets_with_CID'})
processing.run("native:createspatialindex", {'INPUT':'dbname=\'FoodAccess\' port=5432 sslmode=disable key=\'id\' srid=2263 type=MultiPolygon checkPrimaryKeyUnicity=\'0\' table=\"NYCHA_Waste\".\"NYCHA_Developments\" (geom)'})
processing.run("native:createspatialindex", {'INPUT':'dbname=\'FoodAccess\' port=5432 sslmode=disable key=\'id\' srid=2263 type=MultiPolygon checkPrimaryKeyUnicity=\'0\' table=\"NYCHA_Waste\".\"NYCHA_Buildings\" (geom)'})

def get_consolidations(dev_layer_id, cons_tds_att, cons_name_att):
    layers = QgsProject.instance().mapLayers()
    dev_layer = layers[dev_layer_id]
    features = dev_layer.getFeatures()
    
    consolidations = dict()
    
    for feature in features:
        if feature[cons_tds_att] not in consolidations.keys():
            consolidations[feature[cons_tds_att]] = feature[cons_name_att]
    
    return consolidations
    


def get_consolidation_buildings(buildings_layer_id, cons_tds):
    layers = QgsProject.instance().mapLayers()
    layer = layers[buildings_layer_id]
    
    #layer.selectByExpression(f'''"cons_tds"= '{cons_tds}' ''', QgsVectorLayer.SetSelection)
    
    cons_buildings = processing.run("native:extractbyexpression", {'INPUT':layer.dataProvider().dataSourceUri(),
    'EXPRESSION':f''' "cons_tds" = '{cons_tds}' ''',
    'OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_buildings.shp'})['OUTPUT']
    
    new_layer = QgsVectorLayer(cons_buildings, f'{cons_tds}_buildings', "ogr")
    
    return new_layer

def get_consolidation_entrances(entrances_layer_id, cons_tds):
    layers = QgsProject.instance().mapLayers()
    layer = layers[entrances_layer_id]
    
    #layer.selectByExpression(f'''"cons_tds"= '{cons_tds}' ''', QgsVectorLayer.SetSelection)
    
    cons_entrances = processing.run("native:extractbyexpression", {'INPUT':layer.dataProvider().dataSourceUri(),
    'EXPRESSION':f''' "cons_tds" = '{cons_tds}' ''',
    'OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_entrances.shp'})['OUTPUT']
    
    new_layer = QgsVectorLayer(cons_entrances, f'{cons_tds}_entrances', "ogr")
    
    return new_layer

def get_consolidation_developments(dev_layer_id, cons_tds):

    layers = QgsProject.instance().mapLayers()
    layer = layers[dev_layer_id]
    
    #layer.selectByExpression(f'''"cons_tds"= '{cons_tds}' ''', QgsVectorLayer.SetSelection)
    
    cons_devs = processing.run("native:extractbyexpression", {'INPUT':layer.dataProvider().dataSourceUri(),
    'EXPRESSION':f''' "cons_tds" = '{cons_tds}' ''',
    'OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_developments.shp',
    'FAIL_OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_other_developments.shp'})
    
    output_layer = QgsVectorLayer(cons_devs['OUTPUT'], f'{cons_tds}_developments', "ogr")
    fail_output_layer = QgsVectorLayer(cons_devs['FAIL_OUTPUT'], f'{cons_tds}_other_developments', "ogr")
    
    return [output_layer, fail_output_layer]


def get_consolidation_assets(assets_layer_id, cons_tds):
    
    layers = QgsProject.instance().mapLayers()
    layer = layers[assets_layer_id]
    
    cons_assets = processing.run("native:extractbyexpression", {'INPUT':layer.dataProvider().dataSourceUri(),
    'EXPRESSION':f''' "cons_tds" = '{cons_tds}' ''',
    'OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_assets.shp'})['OUTPUT']
    
    new_layer = QgsVectorLayer(cons_assets, f'{cons_tds}_assets', "ogr")
    
    return new_layer

def get_consolidation_recycling(recycling_layer_id, cons_tds):
    
    layers = QgsProject.instance().mapLayers()
    layer = layers[recycling_layer_id]
    
    cons_assets = processing.run("native:extractbyexpression", {'INPUT':layer.dataProvider().dataSourceUri(),
    'EXPRESSION':f''' "cons_tds" = '{cons_tds}' ''',
    'OUTPUT':f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}/{cons_tds}_recycling.shp'})['OUTPUT']
    
    new_layer = QgsVectorLayer(cons_assets, f'{cons_tds}_recycling', "ogr")
    
    return new_layer


def render_map(defining_layer, cons_tds):
    project = QgsProject.instance()
    #Create Layout
    manager = project.layoutManager()       
    layout = QgsPrintLayout(project)        
    layoutName = f"{cons_tds}_layout"
    
    layouts_list = manager.printLayouts()
    for current_layout in layouts_list:
        if current_layout.name() == layoutName:
            manager.removeLayout(current_layout)

    #initializes default settings for blank print layout canvas
    layout.initializeDefaults()  
    
    layout.setName(layoutName)
    manager.addLayout(layout)
    
    map = QgsLayoutItemMap(layout)

    #I have no idea what this does, but it is necessary
    map.setRect(20, 20, 20, 20)
    map.setCrs(defining_layer.crs())
    
    rectangle = defining_layer.extent()
    map.zoomToExtent(rectangle)
    layout.addLayoutItem(map)
    
    map.attemptMove(QgsLayoutPoint(0, 0, QgsUnitTypes.LayoutMillimeters))
    map.attemptResize(QgsLayoutSize(215.9,279.4, QgsUnitTypes.LayoutMillimeters))
    
    # Adding Legend
    legend = QgsLayoutItemLegend(layout)
    
    layersToAdd = ['template_assets_da495e07_720a_40c6_be18_e661edc1036a','recycling_bins_b5449960_727b_4115_b168_c811f4c9fa41', 'NYCHA_Address_Points_20200212_bc5219bf_115c_4010_870c_82862031a0cf', 'template_buildings_72ddfa7a_bfcc_4724_b19f_084b6dcb4cc5','template_developments_5eb69da0_b932_4e62_91dc_fac5a919b767']
    legend_layers = [QgsProject().instance().mapLayers()[layername] for layername in layersToAdd]
    
    root = QgsLayerTree()
    for layer in legend_layers:
        #add layer objects to the layer tree
        root.addLayer(layer)
    legend.model().setRootGroup(root)
    
    root.findLayers()[0].setCustomProperty(value="0,1", key="legend/node-order")
    root.findLayers()[0].setCustomProperty(value="hidden", key="legend/title-style")
    root.findLayers()[0].checked = "Qt::Checked"
    
    layout.addLayoutItem(legend)
    legend.attemptMove(QgsLayoutPoint(154.004,226.012, QgsUnitTypes.LayoutMillimeters))
    
    
    exporter = QgsLayoutExporter(layout)
    image_settings = exporter.ImageExportSettings()
    image_settings.dpi = 300
    image_settings.cropToContents = True
    
    exporter.exportToImage(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/consolidation_maps/{cons_tds}_asset_map.png',image_settings)
    


def generate_consolidation_assetmap(cons_tds, replace=False):
    
    if not os.path.exists(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}'):
        os.makedirs(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}')
    if (not os.path.exists(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/consolidation_maps/{cons_tds}_asset_map.png')) or (replace==True):
        consolidation_development_layers = get_consolidation_developments(DEVS_LAYER_ID, cons_tds)
        consolidation_development_layers[0].loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/consolidation_dev_style_assetmap.qml')
        QgsProject.instance().addMapLayer(consolidation_development_layers[0])
        #QgsProject.instance().addMapLayer(consolidation_development_layers[1])

        consolidation_buildings = get_consolidation_buildings(BUILDINGS_LAYER_ID, cons_tds)
        consolidation_buildings.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/building_style.qml')
        QgsProject.instance().addMapLayer(consolidation_buildings)
        
        consolidation_entrances = get_consolidation_entrances(ENTRANCES_LAYER_ID, cons_tds)
        consolidation_entrances.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/entrances_style.qml')
        QgsProject.instance().addMapLayer(consolidation_entrances)

        consolidation_assets = get_consolidation_assets(ASSETS_LAYER_ID, cons_tds)
        consolidation_assets.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/waste_assets_style.qml')
        QgsProject.instance().addMapLayer(consolidation_assets)
        
        consolidation_recycling_bins = get_consolidation_recycling(RECYCLING_BINS_LAYER_ID, cons_tds)
        consolidation_recycling_bins.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/recycling_bins.qml')
        QgsProject.instance().addMapLayer(consolidation_recycling_bins)
        
        render_map(consolidation_development_layers[0], cons_tds)
        
        for lyr_name in QgsProject.instance().mapLayers().keys():
            if lyr_name not in [BASEMAP_ID, DEVS_LAYER_ID, BUILDINGS_LAYER_ID, 
            ASSETS_LAYER_ID, RECYCLING_BINS_LAYER_ID, ENTRANCES_LAYER_ID, 
            'template_assets_da495e07_720a_40c6_be18_e661edc1036a','template_buildings_72ddfa7a_bfcc_4724_b19f_084b6dcb4cc5',
            'template_developments_5eb69da0_b932_4e62_91dc_fac5a919b767', 'recycling_bins_b5449960_727b_4115_b168_c811f4c9fa41',
            'NYCHA_Address_Points_20200212_bc5219bf_115c_4010_870c_82862031a0cf']:
                QgsProject.instance().removeMapLayer(lyr_name)
    
    else:
        pass
    
    
    
def generate_consolidation_contextmap(cons_tds):
    if not os.path.exists(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}'):
        os.makedirs(f'/Users/kyleslugg/Documents/GIS/NYCHA/Consolidations/{cons_tds}')

    consolidation_development_layers = get_consolidation_developments(DEVS_LAYER_ID, cons_tds)
    consolidation_development_layers[0].loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/consolidation_dev_style_contextmap.qml')
    consolidation_development_layers[1].loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/other_developments_contextmap.qml')
    QgsProject.instance().addMapLayer(consolidation_development_layers[0])
    QgsProject.instance().addMapLayer(consolidation_development_layers[1])

    consolidation_buildings = get_consolidation_buildings(BUILDINGS_LAYER_ID, cons_tds)
    consolidation_buildings.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/building_style.qml')
    QgsProject.instance().addMapLayer(consolidation_buildings)

    consolidation_assets = get_consolidation_assets(ASSETS_LAYER_ID, cons_tds)
    consolidation_assets.loadNamedStyle('/Users/kyleslugg/Documents/GIS/NYCHA/styles/waste_assets_style.qml')
    QgsProject.instance().addMapLayer(consolidation_assets)




consolidations = get_consolidations(DEVS_LAYER_ID, 'cons_tds', 'consolidat')

for cons_tds in consolidations.keys():
#for cons_tds in ['113','111','021','102','023', '073']:
    generate_consolidation_assetmap(cons_tds)
    #print(f"Generated map for consolidation {cons_tds}\n")
#generate_consolidation_assetmap('073')
#print('Done')

    